In [1]:
import gmaps
import pandas as pd
import requests
import json

# Google developer API key
from api_keys import g_key
from pprint import pprint

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
cw_df = pd.read_csv('resources/City_Weather.csv')
cw_df.head()

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Tarascon,4.6603,43.8058,44.31,59,75,15.84
1,Godoy Cruz,-68.8428,-32.9252,70.41,78,40,8.05
2,Acarape,-38.7072,-4.2238,82.36,69,100,6.42
3,Cliftonville,150.9178,-33.4311,72.54,95,3,0.87
4,Ispra,8.6129,45.8148,45.45,75,58,1.01


In [3]:
locs = cw_df[["Latitude", "Longitude"]]
humidity = cw_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, 
                                 dissipating=False,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
good_df = cw_df.loc[(cw_df['Max Temperature'] > 70) & (cw_df['Max Temperature'] < 80) \
                                          & (cw_df['Wind Speed'] < 10) \
                                          & (cw_df['Cloudiness'] < 10)]
good_df

,City,Longitude,Latitude,Max Temperature,Humidity,Cloudiness,Wind Speed
3,Cliftonville,150.9178,-33.4311,72.54,95,3,0.87
21,Kahla,-96.4000,16.9167,74.62,36,0,5.30
387,Silvania,150.6290,-35.1114,70.75,91,6,7.52
423,Sanctuary Point,-72.2042,19.7578,76.84,68,0,9.26
445,Metztitlan,-118.0417,33.9609,72.54,23,0,3.00
453,Serebryansk,-117.5164,33.9925,70.63,17,0,1.99
463,Sarmakovo,150.6167,-33.7667,71.33,91,2,2.53
494,Jingling,2.9386,11.1342,77.77,16,0,5.41
515,St. Thomas,5.1931,7.2526,79.29,23,5,2.80


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : None,
    "radius" : 5000,
    "keyword" : "hotel",
    "key" : g_key
}


In [6]:
coords = []
hotel_info = []
for index, row in good_df.iterrows():
    params['location'] = f"{row['Latitude']},{row['Longitude']}"
    hotel = requests.get(base_url, params=params).json()
    try:
        hotel_info.append({'name' : hotel['results'][0]['name'], 'city' : row['City'].title()})
        coords.append((hotel['results'][0]['geometry']['location']['lat'],hotel['results'][0]['geometry']['location']['lng']))
    except:
        print("Missing field/result... skipping.")
coords

Missing field/result... skipping.
Missing field/result... skipping.


[(-33.4224167, 150.8950806),
 (16.9214202, -96.3631163),
 (-35.1052486, 150.6251818),
 (19.7699627, -72.1979226),
 (33.9745151, -118.0367113),
 (11.1298259, 2.9241925),
 (7.264134299999999, 5.210503000000001)]

In [7]:
hotel_info

[{'name': 'Juniors on Hawkesbury', 'city': 'Cliftonville'},
 {'name': 'Don Cenobio', 'city': 'Kahla'},
 {'name': 'Bay & Basin Boutique Hotel', 'city': 'Silvania'},
 {'name': 'Hotel Satama', 'city': 'Sanctuary Point'},
 {'name': 'DoubleTree by Hilton Whittier Los Angeles', 'city': 'Metztitlan'},
 {'name': 'Hôtel Gb', 'city': 'Jingling'},
 {'name': 'HERITAGE CONTINENTAL HOTEL', 'city': 'St. Thomas'}]

In [8]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
info_box = [info_box_template.format(**hotel) for hotel in hotel_info]
markers = gmaps.marker_layer(coords, info_box_content=info_box)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
print(hotels["results"][0]["name"])
print(hotels["results"][0]["vicinity"])

NameError: name 'hotels' is not defined